## RAG Finetuning LLM
### * **Retrieval based on PDF relevant about fashion recommendation to Vector Database**
### * **Integrating LLM-based attribute aware context with fine-grained fashion retrieval. For each attribute in the query the LLM first generates a detailed attribute-aware context for enriching attribute representations with commonsense business insight requirements.**
### * **The attribute embeddings, enriched with their attribute- aware context, form a conditional query vector that guides the retrieval process, interacting with image patches to focus on relevant regions that match the specified attributes.**
### * **Prompt generation training strategies to enhance its capacity for delivering personalized fashion advice while retaining essential domain knowledge.**
### * **Generative images AI Engineering.**

# LLM Strategies
### These strategies, as reflected in the designed prompts, 
### Ensure that the LLM not only retains its core language processing capabilities but is also finely tuned to analyze and address fashion-related queries with enhanced precision.

## Load & chunk PDF documents

In [4]:
from pypdf import PdfReader

def load_pdf_text(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text

def chunk_text(text, chunk_size=500, overlap=100):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

# call the functions
pdf_text = load_pdf_text("../data/pdf/fashion recommendation LLM.pdf")
pdf_chunks = chunk_text(pdf_text)

In [5]:
pdf_chunks

['Integrating Domain Knowledge into Large Language Models for\nEnhanced Fashion Recommendations\nZhan Shi∗∗\naria2@scu.edu\nSanta Clara University\nSanta Clara, USA\nShanglin Yang†\nkudoysl@gmail.com\nABSTRACT\nFashion, deeply rooted in sociocultural dynamics, evolves as individ-\nuals emulate styles popularized by influencers and iconic figures. In\nthe quest to replicate such refined tastes using artificial intelligence,\ntraditional fashion ensemble methods have primarily used super-\nvised learning to imit',
 'intelligence,\ntraditional fashion ensemble methods have primarily used super-\nvised learning to imitate the decisions of style icons, which falter\nwhen faced with distribution shifts, leading to style replication dis-\ncrepancies triggered by slight variations in input. Meanwhile, large\nlanguage models (LLMs) have become prominent across various\nsectors, recognized for their user-friendly interfaces, strong con-\nversational skills, and advanced reasoning capabilities. T

## Create Text Embeddings

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

model_name = "sentence-transformers/all-MiniLM-L6-v2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
text_model = AutoModel.from_pretrained(model_name)
text_model.eval()

In [ ]:
import warnings
warnings.filterwarnings("ignore")

def embed_texts(texts):
    with torch.no_grad():
        inputs = tokenizer(
            texts, padding=True, truncation=True, return_tensors="pt"
        )
        outputs = text_model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.cpu().numpy()

# call the function
pdf_embeddings = embed_texts(pdf_chunks)
pdf_embeddings.shape

(59, 384)

## Build FAISS Index

In [ ]:
import faiss

dim = pdf_embeddings.shape[1]
index = faiss.IndexFlatL2(dim) # Cosine similarity
faiss.normalize_L2(pdf_embeddings)

index.add(pdf_embeddings)

## Query FAISS

In [ ]:
def search_faiss(query, k=5):
    q_emb = embed_texts([query])
    faiss.normalize_L2(q_emb)
    scores, idxs = index.search(q_emb, k)
    return [pdf_chunks[i] for i in idxs[0]]

"""Create a function to query FAISS index and retrieve relevant contexts based on PDF chunks."""
contexts = search_faiss("What outfits are suitable for a summer wear?")

## RAG with LLM (Context-Aware Generation)
---
## Purpose
* ### Use retrieved context
* ### Prevent hallucination
* ### Generate grounded answers


In [ ]:
torch.__version__

'2.9.1'

In [ ]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

Using device: mps


In [ ]:
# Set environment variables for Mac-Safe, Memory-Safe, Disk-Safe Version

import os

# Put HF cache in temp (auto-cleans, no disk growth)
os.environ["HF_HOME"] = "/tmp/hf_cache"

# Prevent PyTorch MPS from over-allocating unified memory
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.6"

## Load LLM

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

llm_name = "Qwen/Qwen2.5-3B-Instruct"

# bnb_config for 4-bit quantization to prevent memory overload
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(
    llm_name,
    use_fast=True # use fast tokenizer
)

model = AutoModelForCausalLM.from_pretrained(
    llm_name,
    quantization_config=bnb_config,  # 4-bit quantization
    device_map="auto", # uses MPS safely
    low_cpu_mem_usage=True, # avoid duplicate weights in CPU
)

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

RuntimeError: Invalid buffer size: 26.49 GiB

## Build RAG Prompt

In [ ]:
def build_prompt(contexts, question):
    context_block = "\n\n".join(contexts)
    return f"""
You are fashion recommendation assistant expert.

Context:
{context_block}

Question:
{question}

Answer:
"""

## Generate Answer

In [ ]:
def generate_answer(question):
    contexts = search_faiss(question)
    prompt = build_prompt(contexts, question=question)

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outptus = model.generate(
        **inputs,
        max_new_token=200,
        temperature=0.7
    )
    return tokenizer.decode(outptus[0], skip_special_tokens=True)

print(generate_answer("Recommend a summer casual outfit for men"))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

## Auto clean mode saving

In [ ]:
import gc, torch

del model
gc.collect()
torch.mps.empty_cache()